[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/klar74/WS2025_lecture/blob/main/Vorlesung_04/eda_manufacturing.ipynb)

# EDA-Workflow: Schritt für Schritt praktisch üben

In diesem Notebook arbeiten wir systematisch den **8-Punkte EDA-Workflow** aus der Vorlesung ab. Wir nutzen einen simulierten Produktionsdatensatz, um jeden Schritt praktisch zu üben.

**Unser Weg:** Von den Rohdaten zu ersten Hypothesen - systematisch und nachvollziehbar!

## Schritt 1: Ziel festlegen

**Was wollen wir mit dieser Analyse erreichen?**

**Forschungsfrage:** Welche Faktoren beeinflussen die Defektrate in unserer Produktion?

**Erfolgskriterien:**
1. **Hauptziel:** Identifiziere die 2-3 wichtigsten Einflussfaktoren auf Defekte
2. **Nebenziel:** Vergleiche die Lieferanten hinsichtlich Qualität und Energieverbrauch

**Warum ist das wichtig?** 
- Ohne klares Ziel verliert man sich in endlosen Analysen
- Das Ziel bestimmt, welche Diagramme und Kennzahlen relevant sind
- Es hilft bei der späteren Interpretation der Ergebnisse

## Schritt 2: Datenüberblick verschaffen

Zuerst laden wir die Daten und verschaffen uns einen ersten Überblick.

**Wichtig:** In diesem Notebook erstellen wir uns einen **simulierten Datensatz** selbst, anstatt echte Daten von irgendwo zu laden. Das hat Vorteile für das Lernen:
- Wir kennen die "wahren" Zusammenhänge im Datensatz
- Wir können gezielt Datenqualitätsprobleme einbauen
- Der Datensatz ist immer verfügbar und reproduzierbar

In [ ]:
# Bibliotheken laden und Daten erstellen
import pandas as pd          # Für Tabellen (DataFrames)
import matplotlib.pyplot as plt  # Für Grunddiagramme  
import seaborn as sns        # Für schöne statistische Grafiken
import numpy as np           # Für Berechnungen

# Simulierten Produktionsdatensatz erstellen
np.random.seed(42)  # Reproduzierbare Zufallszahlen
n = 500  # 500 Produkte

# Lieferanten: Realistische deutsche Namen statt X, Y, Z
suppliers = np.random.choice(['Maier', 'Schmidt', 'Berger'], n)

# Energieverbrauch abhängig vom Lieferanten (realistische Variation)
energy_means = {'Maier': 95, 'Schmidt': 170, 'Berger': 120}  # Unterschiedliche Mittelwerte
energy = np.array([np.random.normal(energy_means[s], 12, 1)[0] for s in suppliers])

# Defekte: ca. 2% der Produkte sind defekt
defects = np.random.choice([0, 1], n, p=[0.98, 0.02])  # 0=OK, 1=Defekt

# Druck: höher bei defekten Produkten (simulierter Zusammenhang)
pressure = np.random.normal(30, 2, n) + defects * 2.5

# DataFrame erstellen (= Tabelle mit Spalten)
df = pd.DataFrame({
    'ProductID': np.arange(1, n+1),           # Eindeutige Produkt-IDs
    'Temperature': np.random.normal(70, 5, n), # Temperatur normalverteilt
    'Pressure': pressure,                      # Druck (abhängig von Defekten)
    'Defects': defects,                       # Defekt ja/nein
    'Energy': energy,                         # Energieverbrauch (abhängig von Lieferant)
    'Supplier': suppliers                     # Lieferant Maier, Schmidt oder Berger
})

# Nach Lieferant sortieren für bessere Übersicht
df = df.sort_values('Supplier').reset_index(drop=True)

# ABSICHTLICH Datenqualitätsprobleme einbauen für Lernzwecke:
# 1. Duplizierte Zeile hinzufügen
duplicate_row = df.iloc[10].copy()  # Zeile 10 kopieren
df = pd.concat([df, duplicate_row.to_frame().T], ignore_index=True)

# 2. Schreibfehler in Lieferantennamen einbauen
df.loc[5, 'Supplier'] = 'Mayer'   # Schreibfehler: Maier -> Mayer
df.loc[15, 'Supplier'] = 'Mayer'  # Noch ein Schreibfehler

print("✅ Datensatz wurde erstellt!")
print("⚠️  ACHTUNG: Datensatz enthält absichtlich Qualitätsprobleme für Lernzwecke!")
df.head()  # Erste 5 Zeilen anzeigen

In [ ]:
# Grundlegende Informationen über den Datensatz
print("=== DATENÜBERBLICK ===")
print(f"📊 Größe: {df.shape[0]} Zeilen × {df.shape[1]} Spalten")
print(f"📊 Das bedeutet: {df.shape[0]} Produkte mit {df.shape[1]} Merkmalen")
print()

print("=== DATENTYPEN ===")
print(df.dtypes)  # Zeigt Datentypen jeder Spalte
print()

print("=== SPALTEN-INFORMATIONEN ===")
df.info()  # Kompakte Übersicht über den DataFrame

## Schritt 3: Datenqualität prüfen

Bevor wir analysieren, müssen wir prüfen: Sind die Daten "sauber"?

In [ ]:
# Schritt 3a: Fehlende Werte prüfen
print("=== FEHLENDE WERTE ===")
missing_values = df.isnull().sum()  # Anzahl fehlender Werte pro Spalte
print(missing_values)
print(f"✅ Gesamtzahl fehlender Werte: {missing_values.sum()}")
print()

# Schritt 3b: Duplikate prüfen
print("=== DUPLIKATE ===")
duplicate_count = df.duplicated().sum()  # Anzahl doppelter Zeilen
print(f"❌ Anzahl doppelter Zeilen: {duplicate_count}")
if duplicate_count > 0:
    print("   Duplikate gefunden! Diese müssen bereinigt werden.")
    duplicate_rows = df[df.duplicated(keep=False)]  # Alle Duplikate anzeigen
    print("   Duplizierte Zeilen:")
    print(duplicate_rows[['ProductID', 'Supplier', 'Temperature', 'Defects']])
print()

# Schritt 3c: Eindeutige Werte in kategorischen Spalten
print("=== KATEGORISCHE WERTE ===")
print(f"Anzahl Lieferanten: {df['Supplier'].nunique()}")
print(f"Lieferanten: {df['Supplier'].unique()}")
print()

# Detailanalyse der Lieferantennamen
supplier_counts = df['Supplier'].value_counts()
print("Häufigkeit der Lieferantennamen:")
for supplier, count in supplier_counts.items():
    print(f"   {supplier}: {count} mal")
print()

# Logische Schlussfolgerung aus den Häufigkeiten
maier_count = supplier_counts.get('Maier', 0)
mayer_count = supplier_counts.get('Mayer', 0)
print("=== ANALYSE DER SCHREIBWEISEN ===")
print(f"'Maier' kommt {maier_count} mal vor")
print(f"'Mayer' kommt {mayer_count} mal vor")
print()
if mayer_count > 0 and mayer_count < maier_count:
    print("💡 SCHLUSSFOLGERUNG: 'Mayer' ist vermutlich ein Schreibfehler für 'Maier'")
    print("   → Grund: 'Mayer' kommt viel seltener vor als 'Maier'")
    print("   → 'Maier' ist die häufigste Variante und daher vermutlich korrekt")
elif mayer_count > 0:
    print("❓ Unklar: Beide Schreibweisen kommen ähnlich oft vor")
print()

print(f"Defekte: {df['Defects'].value_counts().to_dict()}")  # Häufigkeiten als Dictionary
print()

# Schritt 3d: Datenbereinigung durchführen
print("=== DATENBEREINIGUNG ===")
print("Jetzt beheben wir die gefundenen Probleme:")

# Problem 1: Duplikate entfernen
df_clean = df.drop_duplicates()
removed_duplicates = len(df) - len(df_clean)
print(f"✅ {removed_duplicates} Duplikat(e) entfernt")

# Problem 2: Schreibfehler in Lieferantennamen korrigieren
# Wichtig: .copy() verwenden um SettingWithCopyWarning zu vermeiden
df_clean = df_clean.copy()  # Explizite Kopie erstellen
df_clean['Supplier'] = df_clean['Supplier'].replace('Mayer', 'Maier')
print("✅ Schreibfehler 'Mayer' → 'Maier' korrigiert")

print(f"✅ Bereinigte Lieferanten: {df_clean['Supplier'].unique()}")
print(f"✅ Datensatz verkleinert von {len(df)} auf {len(df_clean)} Zeilen")

# Bereinigte Daten für weitere Analyse verwenden
df = df_clean.copy()
print()
print("🎉 Datenbereinigung abgeschlossen! Weiter mit den sauberen Daten.")

## Schritt 4: Deskriptive Statistik - Verteilungen prüfen

Jetzt schauen wir uns an, wie die Werte in jeder Spalte verteilt sind.

In [ ]:
# Schritt 4a: Statistische Kennzahlen für numerische Spalten
print("=== DESKRIPTIVE STATISTIK ===")
numeric_columns = ['Temperature', 'Pressure', 'Energy']  # Nur numerische Spalten
desc_stats = df[numeric_columns].describe()
print(desc_stats)
print()

# Schritt 4b: Histogramm der Temperaturverteilung
plt.figure(figsize=(8, 4))

# Subplot 1: Temperatur
plt.subplot(1, 2, 1)
plt.hist(df['Temperature'], bins=20, color='steelblue', alpha=0.7)
plt.title('Temperaturverteilung')
plt.xlabel('Temperatur [°C]')
plt.ylabel('Anzahl')

# Subplot 2: Energieverbrauch
plt.subplot(1, 2, 2)
plt.hist(df['Energy'], bins=20, color='orange', alpha=0.7)
plt.title('Energieverbrauch')
plt.xlabel('Energie [kWh]')
plt.ylabel('Anzahl')

plt.tight_layout()  # Layout optimieren
plt.show()

## Schritt 5: Zusammenhänge prüfen

Gibt es Beziehungen zwischen den Variablen? Das ist zentral für unser Ziel: Defekte vorhersagen!

In [ ]:
# Schritt 5a: Scatterplot - Temperatur vs Druck (gefärbt nach Defekten)
plt.figure(figsize=(8, 4))

# Subplot 1: Scatterplot
plt.subplot(1, 2, 1)
# Separate Plots für OK und Defekt damit Legende funktioniert
ok_data = df[df['Defects'] == 0]
defect_data = df[df['Defects'] == 1]
plt.scatter(ok_data['Temperature'], ok_data['Pressure'], c='green', alpha=0.6, label='OK')
plt.scatter(defect_data['Temperature'], defect_data['Pressure'], c='red', alpha=0.6, label='Defekt')
plt.xlabel('Temperatur [°C]')
plt.ylabel('Druck [bar]')
plt.title('Temperatur vs. Druck')
plt.legend()

# Subplot 2: Korrelationsmatrix
plt.subplot(1, 2, 2)
corr_data = df[['Temperature', 'Pressure', 'Energy', 'Defects']].corr()
plt.imshow(corr_data, cmap='coolwarm', aspect='auto')
plt.colorbar()
plt.xticks(range(len(corr_data.columns)), corr_data.columns, rotation=45)
plt.yticks(range(len(corr_data.columns)), corr_data.columns)
plt.title('Korrelationsmatrix')

# Korrelationswerte in die Matrix schreiben
for i in range(len(corr_data.columns)):
    for j in range(len(corr_data.columns)):
        plt.text(j, i, f'{corr_data.iloc[i, j]:.2f}', 
                ha='center', va='center', color='black')

plt.tight_layout()
plt.show()

## Schritt 6: Domänenkontext prüfen

Sind die Werte realistisch? Stimmen sie mit unserem Fachwissen überein?

In [ ]:
# Schritt 6: Plausibilitätsprüfung der Werte
print("=== DOMÄNEN-PLAUSIBILITÄT ===")
print("Prüfen wir, ob die Werte realistisch sind:")
print()

# Temperatur prüfen
temp_min, temp_max = df['Temperature'].min(), df['Temperature'].max()
print(f"🌡️ Temperatur: {temp_min:.1f}°C bis {temp_max:.1f}°C")
if temp_min >= 0 and temp_max <= 200:
    print("   ✅ Realistisch für Industrieprozesse")
else:
    print("   ❌ Unrealistische Werte!")
print()

# Druck prüfen
pressure_min, pressure_max = df['Pressure'].min(), df['Pressure'].max()
print(f"🔧 Druck: {pressure_min:.1f} bis {pressure_max:.1f} bar")
if pressure_min >= 0 and pressure_max <= 100:
    print("   ✅ Realistisch für Produktionsprozesse")
else:
    print("   ❌ Unrealistische Werte!")
print()

# Defektrate prüfen
defect_rate = df['Defects'].mean() * 100
print(f"⚠️ Defektrate: {defect_rate:.1f}%")
if 1 <= defect_rate <= 10:
    print("   ✅ Typisch für industrielle Produktion")
else:
    print("   ❌ Ungewöhnliche Defektrate!")
print()

# Ausreißer in Energy identifizieren
q1 = df['Energy'].quantile(0.25)  # 25%-Quantil
q3 = df['Energy'].quantile(0.75)  # 75%-Quantil
iqr = q3 - q1  # Interquartilsabstand
outlier_threshold = q3 + 1.5 * iqr  # Standardregel für Ausreißer
outliers = df[df['Energy'] > outlier_threshold]
print(f"🔋 Energieverbrauch-Ausreißer: {len(outliers)} Produkte über {outlier_threshold:.1f} kWh")

## Schritt 7: Visualisierungen gezielt nutzen

Jetzt nutzen wir die passenden Diagramme für unsere spezifischen Fragen.

In [ ]:
# Schritt 7: Zielgerichtete Visualisierungen für unsere Forschungsfrage
plt.figure(figsize=(12, 8))

# Subplot 1: Boxplot - Energieverbrauch je Lieferant
plt.subplot(2, 2, 1)
sns.boxplot(x='Supplier', y='Energy', hue='Supplier', data=df, palette='Set2', legend=False)
plt.title('Energieverbrauch je Lieferant')
plt.ylabel('Energie [kWh]')

# Subplot 2: Balkendiagramm - Defektrate je Lieferant
plt.subplot(2, 2, 2)
defect_by_supplier = df.groupby('Supplier')['Defects'].mean() * 100  # In Prozent
defect_by_supplier.plot(kind='bar', color=['red', 'orange', 'yellow'])
plt.title('Defektrate je Lieferant')
plt.ylabel('Defektrate [%]')
plt.xlabel('Lieferant')
plt.xticks(rotation=0)

# Subplot 3: Histogramm - Druck getrennt nach Defekt/OK
plt.subplot(2, 2, 3)
ok_pressure = df[df['Defects'] == 0]['Pressure']      # Druck bei OK-Produkten
defect_pressure = df[df['Defects'] == 1]['Pressure']  # Druck bei Defekten
plt.hist(ok_pressure, bins=15, alpha=0.7, label='OK', color='green')
plt.hist(defect_pressure, bins=15, alpha=0.7, label='Defekt', color='red')
plt.xlabel('Druck [bar]')
plt.ylabel('Anzahl')
plt.title('Druckverteilung: OK vs. Defekt')
plt.legend()

# Subplot 4: Anzahl Produkte je Lieferant
plt.subplot(2, 2, 4)
supplier_counts = df['Supplier'].value_counts()
supplier_counts.plot(kind='pie', autopct='%1.1f%%', colors=['lightblue', 'lightgreen', 'lightcoral'])
plt.title('Verteilung der Produkte je Lieferant')
plt.ylabel('')  # y-Label bei Pie-Chart nicht nötig

plt.tight_layout()
plt.show()

## Schritt 8: Erste Hypothesen notieren

Basierend auf unserer Analyse formulieren wir konkrete Hypothesen für weitere Untersuchungen.

In [ ]:
# Schritt 8: Systematische Auswertung für Hypothesen
print("=== HYPOTHESEN BASIEREND AUF EDA ===")
print()

# Hypothese 1: Druck und Defekte
pressure_corr = df['Pressure'].corr(df['Defects'])
print(f"💡 HYPOTHESE 1: Höherer Druck → Mehr Defekte")
print(f"   Korrelation Druck-Defekte: {pressure_corr:.3f}")
if pressure_corr > 0.1:
    print("   ✅ Unterstützt durch Daten")
else:
    print("   ❌ Nicht unterstützt")
print()

# Hypothese 2: Lieferanten-Unterschiede
print(f"💡 HYPOTHESE 2: Lieferanten unterscheiden sich bei Energie und Defekten")
energy_by_supplier = df.groupby('Supplier')['Energy'].mean()
defects_by_supplier = df.groupby('Supplier')['Defects'].mean() * 100
print("   Energieverbrauch je Lieferant:")
for supplier in ['Berger', 'Maier', 'Schmidt']:  # Alphabetisch sortiert
    print(f"     {supplier}: {energy_by_supplier[supplier]:.1f} kWh, {defects_by_supplier[supplier]:.1f}% Defekte")
print("   ✅ Klare Unterschiede erkennbar")
print()

# Hypothese 3: Temperatur weniger wichtig
temp_corr = df['Temperature'].corr(df['Defects'])
print(f"💡 HYPOTHESE 3: Temperatur hat weniger Einfluss auf Defekte")
print(f"   Korrelation Temperatur-Defekte: {temp_corr:.3f}")
if abs(temp_corr) < 0.1:
    print("   ✅ Bestätigt - schwacher Zusammenhang")
else:
    print("   ❌ Widerspruch - stärkerer Zusammenhang als erwartet")
print()

print("=== NÄCHSTE SCHRITTE ===")
print("✅ Bereit für Modellierung:")
print("   • Druck als wichtigster Prädiktor für Defekte")
print("   • Lieferant als kategorisches Merkmal einbeziehen")
print("   • Temperatur eventuell weglassen")
print("   • Fokus auf Schmidt (höchster Energieverbrauch)")
print()
print("📋 Empfehlung: Beginne mit einfachem Logistic Regression Modell")
print("   Features: Pressure + Supplier")

## 🎉 EDA-Workflow abgeschlossen!

**Was haben wir erreicht?**
- ✅ Klares Ziel definiert und verfolgt
- ✅ Daten systematisch verstanden  
- ✅ Qualitätsprobleme identifiziert **und behoben** (Duplikate, Schreibfehler)
- ✅ Wichtige Muster entdeckt (Druck ↔ Defekte, Lieferanten-Unterschiede)
- ✅ Konkrete Hypothesen für Modellierung entwickelt

**Key Takeaways:**
1. **Druck** ist der stärkste Prädiktor für Defekte
2. **Lieferant Schmidt** verbraucht deutlich mehr Energie
3. **Temperatur** scheint weniger relevant zu sein
4. **Datenbereinigung** war nötig und erfolgreich
5. Datensatz ist jetzt sauber und modelltauglich

**Nächster Schritt:** Mit diesen Erkenntnissen können wir nun ein erstes Vorhersagemodell entwickeln!